In [1]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import ast
from types import SimpleNamespace

In [2]:
# Import custom modules
from GraphBuilder_with_features import create_graph_dataset
from training_utils import train

In [5]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [6]:
# Load data
def load_graph_data(loop):
    """Load graph data from CSV files."""
    edges = []
    y = []
    
    for i in range(loop, loop + 1):
        filename = f'../Graph_Edge_Data/den_graph_data_{loop}.csv'
        df = pd.read_csv(filename)
        edges += df['EDGES'].tolist()
        y += df['COEFFICIENTS'].tolist()
    
    edges = [ast.literal_eval(e) for e in edges]
    graphs_data = list(zip(edges, y))
    return graphs_data

In [7]:
# Define different feature configurations to test
def get_feature_configs():
    """Define different feature configurations for experiments."""
    return {
        'minimal': {
            'selected_features': ['basic', 'face'],
            'laplacian_pe_k': 0
        },
        'planar_focused': {
            'selected_features': ['basic', 'face', 'dual'],
            'laplacian_pe_k': 2
        },
        'balanced': {
            'selected_features': ['basic', 'face', 'spectral_node', 'centrality'],
            'laplacian_pe_k': 3
        },
        'full': {
            'selected_features': ['basic', 'face', 'spectral_node', 'dual', 'centrality'],
            'laplacian_pe_k': 4
        }
    }

In [8]:
# Main experiment configuration
def run_single_experiment(config,dataset ):
    """Run a single experiment with specified configuration."""

    config.in_channels = dataset[0].x.shape[1]
    
    print(f"Dataset created with {len(dataset)} graphs")
    print(f"Feature dimensions: {config.in_channels}")
    print(f"Feature names: {dataset[0].feature_names}")
    
    # Train model
    results = train(config, dataset)
    
    return results

# Experiments

In [9]:
# Load data
graphs_data = load_graph_data(loop=8)

In [15]:
# Standard Configurations 
feat_conf = get_feature_configs()['full']
feat_conf

{'selected_features': ['basic', 'face', 'spectral_node', 'dual', 'centrality'],
 'laplacian_pe_k': 4}

In [ ]:
# Dataset configuration
dataset_config = SimpleNamespace( selected_features=feat_conf,
laplacian_pe_k=3) #Balanced configuration

# Create dataset
dataset, scaler = create_graph_dataset( graphs_data,
    {
        'selected_features': dataset_config.selected_features,
        'laplacian_pe_k': dataset_config.laplacian_pe_k
    }
)

Extracting features...
Normalizing features...
Created dataset with 1432 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']


In [9]:
# Create configuration
config = SimpleNamespace(
    # Model configuration
    model_name='gin',
    hidden_channels=64,
    num_layers=3,
    dropout=0.2,
    
    # Training configuration
    lr=0.034,
    weight_decay=5e-4,
    epochs=70,
    batch_size=32,
    early_stop_patience=20,
    scheduler_type='onecycle',
    
    # WandB configuration
    use_wandb=True,
    project='gnn-planar-graphs_8',
    experiment_name='basic_oneCyleLR',
)

In [ ]:
onecycle_config = SimpleNamespace(
    # Model configuration
    model_name='gin',
    hidden_channels=64,
    num_layers=3,
    dropout=0.2,
    
    # Training configuration
    lr=0.003,  # Reasonable for OneCycleLR
    weight_decay=5e-4,
    epochs=100,
    batch_size=32,
    scheduler_type='onecycle',
    
    # WandB configuration
    use_wandb=True,
    project='gnn-planar-graphs_9_loops',
    experiment_name='gin_onecycle_fixed',
)

In [ ]:
onecycle_config_2 = SimpleNamespace(
    # Model configuration
    model_name='gin',
    hidden_channels=64,
    num_layers=3,
    dropout=0.2,
    
    # Training configuration
    lr=0.03,  # Reasonable for OneCycleLR
    weight_decay=5e-4,
    epochs=100,
    batch_size=32,
    scheduler_type='onecycle',
    
    # WandB configuration
    use_wandb=True,
    project='gnn-planar-graphs_9_loops',
    experiment_name='gin_onecycle_fixed',
)

In [ ]:
# 4. Best practices configuration for small planar graphs
best_practices_config = SimpleNamespace(
    # Model configuration
    model_name='gin',
    hidden_channels=48,  # Not too large for small graphs
    num_layers=2,  # 2-3 layers optimal for small graphs
    dropout=0.15,
    
    # Training configuration
    lr=0.002,
    weight_decay=1e-4,
    epochs=150,
    batch_size=64,  # Larger batch for stability
    scheduler_type='plateau',
    
    # WandB configuration
    use_wandb=True,
    project='gnn-planar-graphs',
    experiment_name='best_practices',
)

In [10]:
results = run_single_experiment(config, dataset )

Dataset created with 1432 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']
Using device: cpu
Train size: 1145, Val size: 287


wandb: Currently logged in as: dian-gabriele (dian-gabriele-desydeutsches-elektronen-synchrotron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Starting training...
Model architecture: gin
Hidden dim: 64, Layers: 3
Initial LR: 0.00408
Epoch   0/70: Train Loss=0.7378, Acc=0.5694, Val Loss=0.6585, Acc=0.5958, LR=0.004628
Epoch  10/70: Train Loss=0.6988, Acc=0.5022, Val Loss=0.6942, Acc=0.4948, LR=0.056805
Epoch  20/70: Train Loss=0.6900, Acc=0.5127, Val Loss=0.6938, Acc=0.4948, LR=0.102000
Epoch  30/70: Train Loss=0.6954, Acc=0.5013, Val Loss=0.6956, Acc=0.5052, LR=0.091818
Epoch  40/70: Train Loss=0.6947, Acc=0.5083, Val Loss=0.6931, Acc=0.5052, LR=0.065424
Epoch  50/70: Train Loss=0.6947, Acc=0.4882, Val Loss=0.6957, Acc=0.4948, LR=0.033301
Epoch  60/70: Train Loss=0.6941, Acc=0.5127, Val Loss=0.6940, Acc=0.4948, LR=0.008208
Epoch  69/70: Train Loss=0.6928, Acc=0.5127, Val Loss=0.6938, Acc=0.4948, LR=0.000000

Best validation accuracy: 0.7178 at epoch 6


current_lr,▁▁▂▂▂▃▃▅▆▆▇█████████▇▇▆▆▆▆▅▅▅▄▄▃▃▂▂▂▂▁▁▁
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇████
train_accuracy,▄▆▇█▆▄▂▂▂▂▁▂▁▁▂▁▂▂▂▁▁▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train_f1,▃▅▇█▇▃▄▅▃▃▃▃▃▄▄▄▁▂▄▃▂▃▄▃▅▂▆▆▆▄▄▆▅▆▆▆▆▆▆▆
train_loss,▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_precision,▅█▅█▇▅▃▂▂▁▂▁▂▁▁▂▁▂▂▂▂▃▁▁▁▂▁▂▂▂▁▂▂▁▂▂▂▂▂▂
train_recall,▂▄▆█▂▅▅▃▃▆▃▃▄▄▄▄▁▄▄▃▃▄▄▆▃▂▇▁▄███████████
val_accuracy,██▇▃▂▂▂▂▂▂▂▂▂▂▁▂▃▃▃▂▂▂▂▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂
val_f1,▆▇███▇▇▇▇▇▇▇▆▇▇▁▁▇▇▇▇▇▇▇▇▇▇▁▇▇▇▇▇▇▇▇▇▇▇▇
val_loss,▁▁▁▁▁▁▁▁█▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,█▇▇▇▁▆▆▆▆▆▆▆▆▆▆▁▆▁▆▆▁▆▆▆▁▆▆▆▁▆▆▆▆▆▆▆▆▆▆▆


In [ ]:
results = run_single_experiment(onecycle_config, dataset )

Dataset created with 13972 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']
Using device: cpu
Train size: 11177, Val size: 2795



Starting training...
Model architecture: gin
Hidden dim: 64, Layers: 3
Initial LR: 0.0003599999999999992
Epoch   0/100: Train Loss=0.5595, Acc=0.6990, Val Loss=0.4834, Acc=0.7463, LR=0.000384
Epoch  10/100: Train Loss=0.4292, Acc=0.7897, Val Loss=0.4383, Acc=0.7696, LR=0.002923
Epoch  20/100: Train Loss=0.4389, Acc=0.7877, Val Loss=0.4353, Acc=0.7839, LR=0.007220
Epoch  30/100: Train Loss=0.4439, Acc=0.7823, Val Loss=0.4180, Acc=0.7835, LR=0.008995
Epoch  40/100: Train Loss=0.4326, Acc=0.7856, Val Loss=0.4144, Acc=0.7914, LR=0.008462
Epoch  50/100: Train Loss=0.4181, Acc=0.7967, Val Loss=0.4060, Acc=0.7857, LR=0.007145
Epoch  60/100: Train Loss=0.3982, Acc=0.8105, Val Loss=0.4036, Acc=0.8043, LR=0.005303
Epoch  70/100: Train Loss=0.3723, Acc=0.8215, Val Loss=0.3721, Acc=0.8132, LR=0.003303
Epoch  80/100: Train Loss=0.3535, Acc=0.8355, Val Loss=0.3499, Acc=0.8240, LR=0.001539
Epoch  90/100: Train Loss=0.3243, Acc=0.8477, Val Loss=0.3387, Acc=0.8329, LR=0.000362
Epoch  99/100: Train Los

current_lr,▁▂▂▄▄▆▆▆▇▇████████▇▇▇▇▇▆▆▅▅▄▄▄▃▃▃▂▂▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇███
train_accuracy,▁▁▂▂▂▁▂▂▂▁▂▂▂▂▁▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▇▇▇▇▇████
train_f1,▁▁▂▃▃▂▃▃▃▃▂▃▃▃▂▃▃▃▂▃▃▃▄▄▄▄▄▅▆▆▆▆▇▇██████
train_loss,█▆▅▅▄▅▄▅▄▅▅▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
train_precision,▂▃▃▄▄▃▃▃▃▂▂▂▁▂▂▃▂▁▃▃▃▄▃▃▃▄▄▅▅▆▆▆▇▇▇█████
train_recall,▁▂▃▃▂▄▄▃▄▂▄▅▄▄▄▃▄▁▂▄▄▄▄▃▃▃▅▅▄▅▆▆▆▇▇▇█▇▇█
val_accuracy,▁▂▃▃▃▄▄▄▃▄▄▄▃▄▁▄▄▄▁▆▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇█▇██
val_f1,▃▆▅▆▆▆▆▄▃▆▆▁▆▆▃▇▅▆▅▇▇▅▆▆▇▆▆▇▇▇▇▇▇███████
val_loss,▇▆▇▅▆▆▇▅▇▆▅▇█▅▇▄▄▆▄▅▃▄▄▄▃▃▂▃▃▃▂▂▂▁▁▁▁▁▁▁
val_precision,▂▃▂▃▄▆▂▅▂▄▂▄▄▁▃▇▅▄▄▃▂▇▅▅▄▅▅▅▇▆▇▆▅▆▇█▇█▇▇
